## Kaggle NLP Tutorial: [Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-2-word-vectors)
-----
# Part 2: Word Vectors
# 1. Introducing Distributed Word Vectors
[Word2vec](https://code.google.com/p/word2vec/), published by Google in 2013, is a neural network implementation that learns [distributed representations](http://www.cs.toronto.edu/~bonner/courses/2014s/csc321/lectures/lec5.pdf) for words.

**1. Word2vec learns quickly relative to other models.**
Other deep or recurrent neural network architectures had been proposed for learning word representations prior to this, but the major problem with these was the long time required to train the models. 

**2. Word2vec does not need labels.** If the network is given enough training data (tens of billions of words), it produces word vectors with intriguing characteristics. 
- Words with similar meanings appear in clusters, and clusters are spaced such that some word relationships, such as analogies, can be reproduced using vector math. 
- The famous example is that, with highly trained word vectors, "king - man + woman = queen."

# 2. Using word2vec in Python
- We use the **gensim** package.
- Although Word2Vec does not require graphics processing units (GPUs) like many deep learning algorithms, it is **compute intensive**. Both Google's version and the Python version rely on multi-threading (running multiple processes in parallel on your computer to save time). 
- To train your model in a reasonable amount of time, you will need **cython**. Word2Vec will run without cython installed, but it will take days to run instead of minutes. (Cython is already included in Anaconda package).

# 3. Preparing to Train a Model
- Word2Vec can learn from unlabeled data.
- We use unlabeledTrain.tsv, which contains 50,000 additional reviews with no labels.

In [2]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "data/kaggle_nlp_labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv( "data/kaggle_nlp_testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv( "data/kaggle_nlp_unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews\n" % (train["review"].size, test["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



### Clean data
Similar to Part 1, but a couple of differences. 
- To train Word2Vec it is better not to remove stop words because the algorithm relies on the broader context of the sentence in order to produce high-quality word vectors. 
- For this reason, we will make stop word removal optional in the functions below. It also might be better not to remove numbers, but we leave that as an exercise for the reader.

In [3]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words, optionally removing stop words.  Returns a list of words.

    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()

    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)

    # 3. Convert words to lower case and split them
    words = review_text.lower().split()

    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    # 5. Return a list of words
    return(words)

### Specify input format: list of lists for Word2Vec
Word2Vec expects single sentences, each one as a list of words -> a list of lists.
- It is not at all straightforward how to split a paragraph into sentences. There are all kinds of gotchas in natural language. 
    - English sentences can end with "?", "!", """, or ".", among other things
    - Spacing and capitalization are not reliable guides either. 
- For this reason, we'll use NLTK's **punkt tokenizer for sentence splitting**. 
    - To use this, you will need to install NLTK and use nltk.download() to download the relevant training file for punkt.

In [4]:
import nltk.data
# nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [5]:
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. 
    # Returns a list of sentences, where each sentence is a list of words

    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

### Break down a review into sentences, and append
**`+=`** vs **`append`** when it comes to **appending a list of lists to another list of lists**
- **`append`**: only append the first list
- **`+=`**: join all of the lists at once

In [7]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["review"]:
    sentences += review_to_sentences(review.decode("utf8"), tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review.decode("utf8"), tokenizer)

/Users/Hongsup/anaconda/lib/python2.7/site-packages/bs4/__init__.py:182: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/Users/Hongsup/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


Parsing sentences from training set
Parsing sentences from unlabeled set

/Users/Hongsup/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Users/Hongsup/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.'

In [8]:
# Check how many sentences we have in total - should be around 850,000+
print len(sentences)

795538


In [10]:
print sentences[0]
print sentences[1]

[u'with', u'all', u'this', u'stuff', u'going', u'down', u'at', u'the', u'moment', u'with', u'mj', u'i', u've', u'started', u'listening', u'to', u'his', u'music', u'watching', u'the', u'odd', u'documentary', u'here', u'and', u'there', u'watched', u'the', u'wiz', u'and', u'watched', u'moonwalker', u'again']
[u'maybe', u'i', u'just', u'want', u'to', u'get', u'a', u'certain', u'insight', u'into', u'this', u'guy', u'who', u'i', u'thought', u'was', u'really', u'cool', u'in', u'the', u'eighties', u'just', u'to', u'maybe', u'make', u'up', u'my', u'mind', u'whether', u'he', u'is', u'guilty', u'or', u'innocent']


# 4. Training and Saving Your Model
## Some [word2vec parameters](http://radimrehurek.com/gensim/models/word2vec.html)

**Architecture**: skip-gram (default) or continuous bag of words 
- We found that skip-gram was very slightly slower but produced better results.

**Training algorithm**: Hierarchical softmax (default) or negative sampling
- Here, default

**Downsampling of frequent words**
- The Google documentation recommends values between .00001 and .001. 
- For us, values closer 0.001 seemed to improve the accuracy of the final model.

**Word vector dimensionality**
- More features result in longer runtimes, and often, but not always, result in better models. 
- Reasonable values can be in the tens to hundreds; we used 300.

**Context / window size**: How many words of context should the training algorithm take into account? 
- 10 seems to work well for hierarchical softmax (more is better, up to a point).

**Worker threads**: Number of parallel processes to run. 
- Computer-specific, but between 4 and 6 should work on most systems.

**Minimum word count**: Limit the size of the vocabulary to meaningful words
- Any word that does not occur at least this many times across all documents is ignored. 
- Reasonable values could be between 10 and 100. 
- In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. 
- This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

## Model fitting

In [14]:
# Import the built-in logging module and configure it so that Word2Vec creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, 
                          size=num_features, min_count = min_word_count, 
                          window = context, sample = downsampling)

Training model...


## Saving the model

In [15]:
# If you don't plan to train the model any further, calling init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and save the model for later use. 
# You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

## Check the status
Mac/Linux: use the "top" command from within Terminal to see if your system is successfully parallelizing while the model is training. With 4 workers, the first process in the list should be Python, and it should show 300-400% CPU usage. 
> `> top -o cpu`


# 5. Exploring the Model Results
## `doesnt_match()`: Deduce which word in a set is most dissimilar from the others:

In [23]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

More subtle difference: countries and cities

In [24]:
model.doesnt_match("france england germany berlin".split())

'berlin'

Not perfect...

In [25]:
model.doesnt_match("paris berlin london austria".split())

'paris'

## **`most_similar()`**: Get insight into the model's word clusters

In [26]:
model.most_similar("man")

[(u'woman', 0.6276901960372925),
 (u'guy', 0.46386197209358215),
 (u'boy', 0.45082786679267883),
 (u'men', 0.43489524722099304),
 (u'lady', 0.4322325885295868),
 (u'person', 0.4219069182872772),
 (u'angus', 0.4098063111305237),
 (u'he', 0.4038394093513489),
 (u'himself', 0.4014739692211151),
 (u'girl', 0.3992985486984253)]

In [27]:
model.most_similar("queen")

[(u'bee', 0.4868798851966858),
 (u'latifah', 0.4850972592830658),
 (u'princess', 0.48380163311958313),
 (u'king', 0.41025078296661377),
 (u'bride', 0.3812709450721741),
 (u'victoria', 0.38001295924186707),
 (u'queens', 0.36199265718460083),
 (u'belle', 0.3581508994102478),
 (u'mrs', 0.3525996506214142),
 (u'blonde', 0.34669917821884155)]

In [28]:
model.most_similar("awful")

[(u'terrible', 0.6613253951072693),
 (u'horrible', 0.6296415328979492),
 (u'dreadful', 0.5971604585647583),
 (u'atrocious', 0.5741333961486816),
 (u'abysmal', 0.5311987400054932),
 (u'horrendous', 0.5225872993469238),
 (u'laughable', 0.5124868750572205),
 (u'horrid', 0.5018720626831055),
 (u'lousy', 0.48499590158462524),
 (u'amateurish', 0.48359769582748413)]

In [29]:
model.most_similar("gay")

[(u'lesbian', 0.5455188751220703),
 (u'homosexual', 0.5112344622612),
 (u'jewish', 0.4181421399116516),
 (u'women', 0.4166848957538605),
 (u'sensitive', 0.40602388978004456),
 (u'male', 0.3952866792678833),
 (u'heterosexual', 0.393507182598114),
 (u'bisexual', 0.3862374424934387),
 (u'queer', 0.38526836037635803),
 (u'community', 0.3794994354248047)]